In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
model = models.densenet201(pretrained=True)
num_classes = 6
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 199MB/s]


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.4602, 0.4453, 0.4321],[0.2245, 0.2198, 0.2210])
])
train_path = datasets.ImageFolder('/content/drive/MyDrive/train', transform=transform)
test_path = datasets.ImageFolder('/content/drive/MyDrive/test', transform=transform)

train_loader = DataLoader(train_path, batch_size=32, shuffle=True)
test_loader = DataLoader(test_path, batch_size=32, shuffle=True)

In [ ]:
critirion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

for epoch in range(5):
  model.train()
  running_loss = 0.0

  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = critirion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")
  torch.save(model.state_dict(), 'model.pth')


Epoch 1 - Loss: 0.6465217388020112
Epoch 2 - Loss: 0.16007544931310874
Epoch 3 - Loss: 0.10443089974041168
Epoch 4 - Loss: 0.07099323680337805


In [ ]:
# Testing and loading
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
img_path = '/content/drive/MyDrive/test/Pitted/PS_10.bmp'
image = Image.open(img_path).convert('RGB')
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

image_tensor = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    outputs = model(image_tensor)
    probabilities = F.softmax(outputs, dim=1)
    confidence, predicted_class_idx = torch.max(probabilities, 1)

    class_name = train_path.classes
    predicted_label = class_name[predicted_class_idx.item()]
    confidence_score = confidence.item()
    print(f"Predicted Lable: {predicted_label}({confidence_score:.2f}% Confidence)")
    plt.imshow(image)
    plt.title(f"predicted: {predicted_label}")
    plt.axis('off')
    plt.show()


